In [7]:
import pygal
from IPython.display import HTML

In [22]:
from sqlalchemy import create_engine  
engine=create_engine('sqlite:///ganji_zufang.sqlite',echo=True)  

from sqlalchemy import Table,Column,Integer,String,ForeignKey  
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()
DBSession = sessionmaker(bind=engine)
session = DBSession()
class ZuFang(Base):
    __tablename__ = "zufang"
    id = Column(Integer, primary_key=True)
    title = Column(String(400))
    rent = Column(String(400))
    info = Column(String(400))
    address = Column(String(400))
    
    @classmethod
    def save(cls, data):
        session.add(data)
        session.commit()
        return data.id

In [13]:
hhzx = session.query(ZuFang).filter(ZuFang.info.contains("豪华装修")).count()
jzx = session.query(ZuFang).filter(ZuFang.info.contains("精装修")).count()
jdzx = session.query(ZuFang).filter(ZuFang.info.contains("简单装修")).count()
mp = session.query(ZuFang).filter(ZuFang.info.contains("毛坯")).count()

2018-09-04 00:20:25,598 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT zufang.id AS zufang_id, zufang.title AS zufang_title, zufang.rent AS zufang_rent, zufang.info AS zufang_info, zufang.address AS zufang_address 
FROM zufang 
WHERE (zufang.info LIKE '%' || ? || '%')) AS anon_1
2018-09-04 00:20:25,600 INFO sqlalchemy.engine.base.Engine ('豪华装修',)
2018-09-04 00:20:25,616 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT zufang.id AS zufang_id, zufang.title AS zufang_title, zufang.rent AS zufang_rent, zufang.info AS zufang_info, zufang.address AS zufang_address 
FROM zufang 
WHERE (zufang.info LIKE '%' || ? || '%')) AS anon_1
2018-09-04 00:20:25,617 INFO sqlalchemy.engine.base.Engine ('精装修',)
2018-09-04 00:20:25,637 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT zufang.id AS zufang_id, zufang.title AS zufang_title, zufang.rent AS zufang_rent, zufang.info AS zufang_info, zufang.address AS zufang_address 
F

In [17]:
lineh_chart = pygal.HorizontalBar()
lineh_chart.title = '装修类型'
lineh_chart.add('豪华装修 {}'.format(hhzx), hhzx)
lineh_chart.add('精装修 {}'.format(jzx), jzx)
lineh_chart.add('简单装修 {}'.format(jdzx), jdzx)
HTML(lineh_chart.render().decode())

In [19]:
zz = session.query(ZuFang).filter(ZuFang.info.contains("整租")).count()
zw = session.query(ZuFang).filter(ZuFang.info.contains("主卧")).count()
cw = session.query(ZuFang).filter(ZuFang.info.contains("次卧")).count()

2018-09-04 00:29:17,600 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT zufang.id AS zufang_id, zufang.title AS zufang_title, zufang.rent AS zufang_rent, zufang.info AS zufang_info, zufang.address AS zufang_address 
FROM zufang 
WHERE (zufang.info LIKE '%' || ? || '%')) AS anon_1
2018-09-04 00:29:17,602 INFO sqlalchemy.engine.base.Engine ('整租',)
2018-09-04 00:29:17,627 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT zufang.id AS zufang_id, zufang.title AS zufang_title, zufang.rent AS zufang_rent, zufang.info AS zufang_info, zufang.address AS zufang_address 
FROM zufang 
WHERE (zufang.info LIKE '%' || ? || '%')) AS anon_1
2018-09-04 00:29:17,630 INFO sqlalchemy.engine.base.Engine ('主卧',)
2018-09-04 00:29:17,645 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT zufang.id AS zufang_id, zufang.title AS zufang_title, zufang.rent AS zufang_rent, zufang.info AS zufang_info, zufang.address AS zufang_address 
FROM

In [21]:
pie_chart = pygal.Pie()
pie_chart.title = '租房状况'
pie_chart.add('整租{}'.format(zz),zz)
pie_chart.add('主卧{}'.format(zw),zw)
pie_chart.add('次卧{}'.format(cw),cw)
HTML(pie_chart.render().decode())

# 深入分析

现在的数据还是不规整，接下来转成dataframe格式，找出unique值，进行更多分析

In [25]:
import pandas as pd

In [39]:
sql_cmd = "SELECT * FROM ZuFang"
df = pd.read_sql(sql_cmd, con = engine)

2018-09-04 00:51:33,673 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("SELECT * FROM ZuFang")
2018-09-04 00:51:33,675 INFO sqlalchemy.engine.base.Engine ()
2018-09-04 00:51:33,679 INFO sqlalchemy.engine.base.Engine SELECT * FROM ZuFang
2018-09-04 00:51:33,680 INFO sqlalchemy.engine.base.Engine ()


In [70]:
df.head()

,title,rent,info,address
6,地铁6号线北运河西附近运河园的两居家电家具全齐出租,4000 元/月,整租 2室1厅1卫 95㎡ 南北向 低层（共6层）\n 简单装修,通州 武夷花园租房 运河园
7,沙河地铁 五福家园精装 两居室 随时入住 民水民电 包物业供,3700 元/月,整租 2室1厅1卫 85㎡ 南北向 中层（共6层）\n 精装修,昌平 沙河镇租房 五福家园
8,西湖新村南北朝向大三居出租,5500 元/月,整租 3室2厅1卫 137㎡ 南北向 高层（共6层）\n 精装修,昌平 北七家租房
9,领地OFFICE 3室1厅 一层带半地下 可做教育,15500 元/月,整租 3室1厅1卫 170㎡ 南北向 低层（共11层）\n 精装修,朝阳 北苑租房 领地OFFICE
10,宏福苑 电梯 大三居两卫6000 精装修 全齐 预约看房,6000 元/月,整租 3室2厅2卫 130㎡ 南北向 中层（共9层）\n 精装修,昌平 北七家租房 宏福苑


In [66]:
len(df)

16638

### 区域

In [ ]:
region = 

# 小节

#### 知识点：
- pygal、cairosvg、cairo第三方库文件
- sqlalchemy 查询作用

- 晚上脚本爬虫